# Symmetric Encryption
Excerpt from *[Symmetric-key algorithm](https://en.wikipedia.org/wiki/Symmetric-key_algorithm) on Wikipedia*:

Symmetric-key algorithms are algorithms for cryptography that use the same cryptographic keys for both the encryption of plaintext and the decryption of ciphertext. The keys may be identical, or there may be a simple transformation to go between the two keys.

The keys, in practice, represent a shared secret between two or more parties that can be used to maintain a private information link. The requirement that both parties have access to the secret key is one of the main drawbacks of symmetric-key encryption, in comparison to public-key encryption (also known as asymmetric-key encryption).

However, symmetric-key encryption algorithms are usually better for bulk encryption. Except for the one-time pad, they have a smaller key size, which means less storage space and faster transmission. Due to this, asymmetric-key encryption is often used to exchange the secret key for symmetric-key encryption.

## AES Encryption
The classic encryption from the early 2000's, estbalished by US NIST in 2001.

Excerpt *from [Advanced Encryption Standard](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) on Wikipedia.*:

AES is a variant of the Rijndael block cipher[5] developed by two Belgian cryptographers, Joan Daemen and Vincent Rijmen, who submitted a proposal[7] to NIST during the AES selection process.[8] Rijndael is a family of ciphers with different key and block sizes. For AES, NIST selected three members of the Rijndael family, each with a block size of 128 bits, but three different key lengths: 128, 192 and 256 bits.

AES has been adopted by the U.S. government. It supersedes the Data Encryption Standard (DES), which was published in 1977. The algorithm described by AES is a symmetric-key algorithm, meaning the same key is used for both encrypting and decrypting the data.

In the United States, AES was announced by the NIST as U.S. FIPS PUB 197 (FIPS 197) on November 26, 2001. This announcement followed a five-year standardization process in which fifteen competing designs were presented and evaluated, before the Rijndael cipher was selected as the most suitable.

AES is included in the ISO/IEC 18033-3 standard. AES became effective as a U.S. federal government standard on May 26, 2002, after approval by U.S. Secretary of Commerce Donald Evans. AES is available in many different encryption packages, and is the first (and only) publicly accessible cipher approved by the U.S. National Security Agency (NSA) for top secret information when used in an NSA approved cryptographic module.

In [5]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend
import os
import time
import statistics


class SymmetricEncrypter_AES:
    def __init__(self):
        pass

    def derive_key(self, password: str, salt: bytes) -> bytes:
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # AES-256 requires a 256-bit (32-byte) key
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )
        return kdf.derive(password.encode())

    def encrypt(self, data: str, password: str) -> (bytes, bytes, bytes):
        salt = os.urandom(16)  # Generate a random salt
        key = self.derive_key(password, salt)  # Derive the key
        iv = os.urandom(16)  # AES requires a 16-byte IV

        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        padder = PKCS7(algorithms.AES.block_size).padder()
        padded_data = padder.update(data.encode()) + padder.finalize()
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()

        return ciphertext, salt, iv

    def decrypt(self, ciphertext: bytes, password: str, salt: bytes, iv: bytes) -> str:
        key = self.derive_key(password, salt)
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()

        padded_data = decryptor.update(ciphertext) + decryptor.finalize()
        unpadder = PKCS7(algorithms.AES.block_size).unpadder()
        data = unpadder.update(padded_data) + unpadder.finalize()

        return data.decode()


def run_standard_test():
    encrypter = SymmetricEncrypter_AES()
    key_string = "strong password"

    test_sizes = [100, 1000, 10000, 100000]
    epochs = 3

    for size in test_sizes:
        data = "A" * size
        print(f"\nTesting with {size} characters:")

        encrypt_times = []
        decrypt_times = []

        for epoch in range(epochs):
            start_time = time.time()
            ciphertext, salt, iv = encrypter.encrypt(data, key_string)
            encrypt_time = time.time() - start_time
            encrypt_times.append(encrypt_time)

            start_time = time.time()
            decrypted_data = encrypter.decrypt(ciphertext, key_string, salt, iv)
            decrypt_time = time.time() - start_time
            decrypt_times.append(decrypt_time)

            print(f"Epoch {epoch + 1}:")
            print(f"Encryption time: {encrypt_time:.4f} seconds")
            print(f"Decryption time: {decrypt_time:.4f} seconds")
            print(f"Total time: {encrypt_time + decrypt_time:.4f} seconds")

        avg_encrypt = statistics.mean(encrypt_times)
        avg_decrypt = statistics.mean(decrypt_times)
        print(f"\nAverage encryption time: {avg_encrypt:.4f} seconds")
        print(f"Average decryption time: {avg_decrypt:.4f} seconds")
        print(f"Average total time: {avg_encrypt + avg_decrypt:.4f} seconds")
        print(f"Message size: {len(ciphertext)} bytes")


run_standard_test()



Testing with 100 characters:
Epoch 1:
Encryption time: 0.0557 seconds
Decryption time: 0.0520 seconds
Total time: 0.1076 seconds
Epoch 2:
Encryption time: 0.0549 seconds
Decryption time: 0.0595 seconds
Total time: 0.1144 seconds
Epoch 3:
Encryption time: 0.0770 seconds
Decryption time: 0.0556 seconds
Total time: 0.1327 seconds

Average encryption time: 0.0625 seconds
Average decryption time: 0.0557 seconds
Average total time: 0.1182 seconds
Message size: 112 bytes

Testing with 1000 characters:
Epoch 1:
Encryption time: 0.0505 seconds
Decryption time: 0.0533 seconds
Total time: 0.1038 seconds
Epoch 2:
Encryption time: 0.0542 seconds
Decryption time: 0.0577 seconds
Total time: 0.1119 seconds
Epoch 3:
Encryption time: 0.0523 seconds
Decryption time: 0.0544 seconds
Total time: 0.1067 seconds

Average encryption time: 0.0523 seconds
Average decryption time: 0.0551 seconds
Average total time: 0.1074 seconds
Message size: 1008 bytes

Testing with 10000 characters:
Epoch 1:
Encryption time: 